In [ ]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)])
  except RuntimeError as e:
    print(e)

In [1]:
X_sentence=list()
Y_sentence=list()
count=0
base_path="dataset"
X_train=list()
Y_train=list()
X_val=list()
Y_val=list()
X_test=list()
Y_test=list()
with open(base_path+"/S1-test.txt", encoding="utf8") as file:
    for line in file:
        if(line!='\n'):
            X_sentence.append(line.split(" ")[0])
            Y_sentence.append(line.split(" ")[1][:-1])
        else:
            count+=1
            if(count==2):
                X_test.append(X_sentence)      
                Y_test.append(Y_sentence)        
                X_sentence=list()
                Y_sentence=list()
                count=0
with open(base_path+"/S2-train.txt", encoding="utf8") as file:
    for line in file:
        if(line!='\n'):
            X_sentence.append(line.split(" ")[0])
            Y_sentence.append(line.split(" ")[1][:-1])
        else:
            count+=1
            if(count==2):
                X_train.append(X_sentence)      
                Y_train.append(Y_sentence)        
                X_sentence=list()
                Y_sentence=list()
                count=0
with open(base_path+"/S3-val.txt", encoding="utf8") as file:
    for line in file:
        if(line!='\n'):
            X_sentence.append(line.split(" ")[0])
            Y_sentence.append(line.split(" ")[1][:-1])
        else:
            count+=1
            if(count==2):
                X_val.append(X_sentence)      
                Y_val.append(Y_sentence)        
                X_sentence=list()
                Y_sentence=list()
                count=0
# sentences it is a list of sentences. Every sentence is a list of tuple
tokenized_sentences=X_train+X_test+X_val
tokenized_labels=Y_train+Y_test+Y_val

In [ ]:
c=0
for sent in Y_train:
    for el in sent:
        if len(el)>2 and el[2:]=='BIOP':
            print(el)

In [2]:
from gensim.models import FastText

model_2=FastText(sentences=tokenized_sentences, vector_size=100, window=10, min_count=5, sg=1, negative=10, alpha=0.01, sample=0.0001)

In [3]:
model_2.train(tokenized_sentences, total_examples=len(tokenized_sentences), epochs=20)

(3047698, 6991040)

In [21]:
from modelli.wrapper import Sequence
import modelli.wrapper
import importlib
importlib.reload(modelli.wrapper)

lstm=Sequence(embeddings=model_2.wv, initial_vocab=model_2.wv.key_to_index)
lstm.fit_vocab(X_train,Y_train)

In [22]:
import tensorflow as tf
lstm.model.call((tf.keras.Input(shape=[17]),tf.keras.Input(shape=[17,34])),training=False)


word_embedding KerasTensor(type_spec=TensorSpec(shape=(None, 17, 100), dtype=tf.float32, name=None), name='word_embedding/embedding_lookup/Identity_1:0', description="created by layer 'word_embedding'")
char_embedding KerasTensor(type_spec=TensorSpec(shape=(None, 17, 34, 25), dtype=tf.float32, name=None), name='char_embedding/embedding_lookup/Identity_1:0', description="created by layer 'char_embedding'")
char_blstm KerasTensor(type_spec=TensorSpec(shape=(None, 17, 50), dtype=tf.float32, name=None), name='time_distributed_7/Reshape_2:0', description="created by layer 'time_distributed_7'")
concatenation KerasTensor(type_spec=TensorSpec(shape=(None, 17, 150), dtype=tf.float32, name=None), name='concatenate_7/concat:0', description="created by layer 'concatenate_7'")
dropout KerasTensor(type_spec=TensorSpec(shape=(None, 17, 150), dtype=tf.float32, name=None), name='dropout_7/Identity:0', description="created by layer 'dropout_7'")
bilstm KerasTensor(type_spec=TensorSpec(shape=(None, 17, 

<KerasTensor: shape=(None, 17, 49) dtype=float32 (created by layer 'crf_7')>

In [ ]:
from modelli.trainer import Trainer

trainer = Trainer(lstm.model, preprocessor=lstm.p)
trainer.train(X_train, Y_train, X_val, Y_val, epochs=1, batch_size=32, verbose=1, shuffle=True)


In [ ]:
from keras.backend import batch_get_value
import pickle

lstm.save(model_json="model_json", model_file="model", preprocessor_file="preprocessor")
# symbolic_weights = lstm.model.optimizer.variables()
# weight_values = batch_get_value(symbolic_weights)
# with open('optimizer.pkl', 'wb') as f:
 #    pickle.dump(symbolic_weights, f)


In [ ]:
import json

model_json=lstm.model.to_json()
with open('model_json', 'w') as f:
    f.write(json.dumps(model_json))


In [ ]:
from modelli.wrapper import Sequence

lstm=Sequence.load(model_json="model_json", model_folder="model", preprocessor_file="preprocessor")
# lstm.model.optimizer.set_weights()
# with open('optimizer.pkl', 'rb') as f:
  #   weight_values = pickle.load(f)
# lstm.model.optimizer.set_weights(weight_values)

In [4]:
from modelli.layers import CRF
lstm.model.compile(loss=CRF(lstm.model._num_labels).loss_function, optimizer=lstm.model.optimizer)

AttributeError: 'Bi' object has no attribute '_num_labels'

In [8]:
import pickle
pickle.dump(lstm, open('wrapper.pkl', 'wb'))

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...bilstm
......backward_layer
.........cell
............vars
...............0
...............1
...............2
.........vars
......forward_layer
.........cell
............vars
...............0
...............1
...............2
.........vars
......layer
.........cell
............vars
.........vars
......vars
...char_bilstm
......layer
.........backward_layer
............cell
...............vars
..................0
..................1
..................2
............vars
.........forward_layer
............cell
...............vars
..................0
..................1
..................2
............vars
.........layer
............cell
...............vars
............vars
.........vars
......vars
...char_embeddings
......vars
.........0
...concat
......vars
...dense
......vars
.........0
.........1
...dropout
......vars
...last_layer
......vars
.........0
.........1
.........2
.........3
.........4
...layers
......embed

In [9]:
pickled_model = pickle.load(open('wrapper.pkl', 'rb'))

Keras model archive loading:
File Name                                             Modified             Size
variables.h5                                   2023-03-21 12:01:02      6163908
metadata.json                                  2023-03-21 12:01:02           64
config.json                                    2023-03-21 12:01:02          456


TypeError: Unable to revive model from config. When overriding the `get_config()`, make sure that the returned config contains all items used as arguments in the constructor to <class 'modelli.models.Bi'>, which is the default behavior. You can override this default behavior by defining a `from_config` method to specify how to create an instance of Bi from the config. 

Error encountered during deserialization:
Bi.__init__() missing 2 required positional arguments: 'num_labels' and 'word_vocab_size'